In [1]:
pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, DoubleType, BooleanType
from pyspark.sql import functions as F
from datetime import datetime

In [3]:
def create_spark_session():
    spark = SparkSession.builder.appName('Formula1').getOrCreate()
    return spark

In [4]:
spark_circuits = create_spark_session()
df_circuits = spark_circuits.read.csv('/content/circuits.csv', header=True, inferSchema=True)


In [5]:
spark_races = create_spark_session()
df_races = spark_races.read.csv('/content/races.csv', header=True, inferSchema=True)

In [6]:
spark_results = create_spark_session()
df_results = spark_results.read.csv('/content/results.csv', header=True, inferSchema=True)

In [7]:
spark_drivers = create_spark_session()
df_drivers = spark_drivers.read.csv('/content/drivers.csv', header=True, inferSchema=True)

In [8]:
spark_constructors = create_spark_session()
df_constructors = spark_constructors.read.csv('/content/constructors.csv', header=True, inferSchema=True)

In [9]:
spark_qualifying = create_spark_session()
df_qualifying = spark_qualifying.read.csv('/content/qualifying.csv', header=True, inferSchema=True)


In [10]:
print('circuits')
df_circuits.printSchema()
print('races')
df_races.printSchema()
print('results')
df_results.printSchema()
print('drivers')
df_drivers.printSchema()
print('constructors')
df_constructors.printSchema()
print('qualifying')
df_qualifying.printSchema()

circuits
root
 |-- circuitId: integer (nullable = true)
 |-- circuitRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- url: string (nullable = true)

races
root
 |-- raceId: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- round: integer (nullable = true)
 |-- circuitId: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- url: string (nullable = true)
 |-- fp1_date: string (nullable = true)
 |-- fp1_time: string (nullable = true)
 |-- fp2_date: string (nullable = true)
 |-- fp2_time: string (nullable = true)
 |-- fp3_date: string (nullable = true)
 |-- fp3_time: string (nullable = true)
 |-- quali_date: string (nullable = true)
 |-- quali_time: string (nullable = true)
 |-- sprint_d

1. Qual equipe teve o maior número de vitórias em cada temporada nos últimos 5 nos?

In [11]:
# PRIMEIRA QUESTAO DE NEGOCIO
current_year = datetime.now().year

last_five_races = df_races.filter(F.col('year') >= current_year - 5)

join_df = last_five_races.alias('r').join(df_results.alias('res'), F.col('r.raceId') == F.col('res.raceId')).join(df_constructors.alias('c'), F.col('res.constructorId') == F.col('c.constructorId'))

winners_df = join_df.filter(F.col('position') == 1)

group_df = winners_df.groupBy('r.year', 'c.name').agg(F.count('*').alias('wins'))

result_df = group_df.orderBy(F.col('year').asc(), F.col('wins').desc())

result_df.show()

+----+--------------+----+
|year|          name|wins|
+----+--------------+----+
|2019|      Mercedes|  15|
|2019|       Ferrari|   3|
|2019|      Red Bull|   3|
|2020|      Mercedes|  13|
|2020|      Red Bull|   2|
|2020|    AlphaTauri|   1|
|2020|  Racing Point|   1|
|2021|      Red Bull|  11|
|2021|      Mercedes|   9|
|2021|       McLaren|   1|
|2021|Alpine F1 Team|   1|
|2022|      Red Bull|  17|
|2022|       Ferrari|   4|
|2022|      Mercedes|   1|
|2023|      Red Bull|  21|
|2023|       Ferrari|   1|
|2024|      Red Bull|   7|
|2024|       McLaren|   2|
|2024|       Ferrari|   2|
|2024|      Mercedes|   2|
+----+--------------+----+



2.Qual piloto acumulou o maior número de voltas mais rápidas em corridas desde o ano 2000?

In [12]:
join_df = df_results.alias('res') \
    .join(df_drivers.alias('d'), F.col('res.driverId') == F.col('d.driverId')) \
    .join(df_races.alias('r'), (F.col('res.raceId') == F.col('r.raceId')) & (F.col('r.year') >= 2000))

fastest_laps_df = join_df.filter(F.col('res.rank') == 1)

group_df = fastest_laps_df.groupBy('d.driverRef').agg(F.count('*').alias('fastest_laps'))

result_df = group_df.orderBy(F.col('fastest_laps').desc()).limit(1)

result_df.show()

+---------+------------+
|driverRef|fastest_laps|
+---------+------------+
| hamilton|          66|
+---------+------------+



3. Qual foi o piloto que mais vezes conquistou uma posição no pódio?

In [13]:
joined_df = df_drivers.alias('d') \
    .join(df_results.alias('res'), F.col('d.driverId') == F.col('res.driverId'))

podium_df = joined_df.filter(F.col('res.position') <= 3)

grouped_df = podium_df.groupBy('d.forename', 'd.surname').agg(F.count('*').alias('podiums'))

result_df = grouped_df.orderBy(F.col('podiums').desc()).limit(1)

result_df.show()

+--------+--------+-------+
|forename| surname|podiums|
+--------+--------+-------+
|   Lewis|Hamilton|    200|
+--------+--------+-------+



4. Quais foram os circuitos que mais apareceram no calendário da F1 nos últimos 30 anos?

In [14]:
current_year = datetime.now().year

last30years_races = df_races.filter(F.col('year') >= current_year - 30)

join_df = last30years_races.alias('r') \
    .join(df_circuits.alias('c'), F.col('r.circuitId') == F.col('c.circuitId'))

group_df = join_df.groupBy('c.name').agg(F.count('*').alias('appearances'))

result_df = group_df.orderBy(F.col('appearances').desc())

result_df.show()

+--------------------+-----------+
|                name|appearances|
+--------------------+-----------+
| Silverstone Circuit|         32|
|Autodromo Naziona...|         31|
|Circuit de Barcel...|         31|
|         Hungaroring|         31|
|Autódromo José Ca...|         30|
|   Circuit de Monaco|         30|
|Circuit de Spa-Fr...|         29|
|Circuit Gilles Vi...|         28|
|Albert Park Grand...|         27|
|      Suzuka Circuit|         27|
|Bahrain Internati...|         21|
|      Hockenheimring|         20|
|       Red Bull Ring|         20|
|Sepang Internatio...|         19|
|Autodromo Enzo e ...|         17|
|Shanghai Internat...|         17|
|         Nürburgring|         17|
|  Yas Marina Circuit|         16|
|Circuit de Nevers...|         15|
|Marina Bay Street...|         15|
+--------------------+-----------+
only showing top 20 rows



5. Qual a distribuição de pódios entre as equipes nos últimos 15 anos?

In [15]:
current_year = datetime.now().year

last15years_races = df_races.filter(F.col('year') >= current_year - 15)

join_df = df_results.alias('res') \
    .join(df_constructors.alias('c'), F.col('res.constructorId') == F.col('c.constructorId')) \
    .join(last15years_races.alias('r'), F.col('res.raceId') == F.col('r.raceId'))

podium_df = join_df.filter(F.col('res.position') <= 3)

group_df = podium_df.groupBy('c.name').agg(F.count('*').alias('podiums'))

result_df = group_df.orderBy(F.col('podiums').desc())

result_df.show()

+--------------+-------+
|          name|podiums|
+--------------+-------+
|      Mercedes|    277|
|      Red Bull|    274|
|       Ferrari|    195|
|       McLaren|     83|
|      Lotus F1|     25|
|      Williams|     17|
|         Brawn|     15|
|       Renault|      9|
|  Aston Martin|      9|
|   Force India|      6|
|        Toyota|      5|
|Alpine F1 Team|      4|
|        Sauber|      4|
|  Racing Point|      4|
|    Toro Rosso|      2|
|    BMW Sauber|      2|
|    AlphaTauri|      2|
+--------------+-------+

